In [ ]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: 

In [ ]:
#%matplotlib inline
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

In [ ]:
# Number of workers for dataloader
workers = 2
# Batch size during training
batch_size = 128
# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 64
# Number of channels in the training images. For color images this is 3
nc = 3
# Size of z latent vector (i.e. size of generator input)
nz = 100
# Size of feature maps in generator
ngf = 64
# Number of GPUs available. Use 0 for CPU mode.
ngpu = 1

In [ ]:
# Generator Code

class Generator(nn.Module):
  def __init__(self, ngpu):
      super(Generator, self).__init__()
      self.ngpu = ngpu
      self.main = nn.Sequential(
          # input is Z, going into a convolution
          nn.ConvTranspose2d( nz, ngf * 8, 4, 1, 0, bias=False),
          nn.BatchNorm2d(ngf * 8),
          nn.ReLU(True),
          # state size. ``(ngf*8) x 4 x 4``
          nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
          nn.BatchNorm2d(ngf * 4),
          nn.ReLU(True),
          # state size. ``(ngf*4) x 8 x 8``
          nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),
          nn.BatchNorm2d(ngf * 2),
          nn.ReLU(True),
          # state size. ``(ngf*2) x 16 x 16``
          nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False),
          nn.BatchNorm2d(ngf),
          nn.ReLU(True),
          # state size. ``(ngf) x 32 x 32``
          nn.ConvTranspose2d( ngf, nc, 4, 2, 1, bias=False),
          nn.Tanh()
          # state size. ``(nc) x 64 x 64``
      )

  def forward(self, input):
      return self.main(input)

In [ ]:
# Decide which device we want to run on
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

In [ ]:
gen_path = '/content/drive/MyDrive/CS490/DCGAN - 10K images 60 epoch/gen_weights.pth'

In [ ]:
gen = Generator(ngpu).to(device)
gen.load_state_dict(torch.load(gen_path))

<All keys matched successfully>

# **Loading 100 real images**

In [ ]:
! pip install kaggle

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d thesighsrikar/concrete-crack-images-for-classification

 99% 231M/234M [00:02<00:00, 113MB/s]
100% 234M/234M [00:02<00:00, 101MB/s]


In [ ]:
!unzip -q concrete-crack-images-for-classification.zip

In [ ]:
import shutil

# Source and destination paths
source_folder_positive = "/content/Concrete/Positive"
destination_folder = "/content/root/REAL"

# Move positive folder to root folder
shutil.move(source_folder_positive, destination_folder)

'/content/root/REAL'

In [ ]:
#Number of images in folder
import os

def count_images_in_directory(directory_path):
    image_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff']  # Add more extensions if needed
    image_count = 0

    for file_name in os.listdir(directory_path):
        if any(file_name.lower().endswith(ext) for ext in image_extensions):
            image_count += 1

    return image_count

# Example usage:
directory_path = destination_folder  # Change this to your directory path
image_count = count_images_in_directory(directory_path)
print(f"Number of images in {directory_path}: {image_count}")


Number of images in /content/root/REAL: 100


In [ ]:
#Randomly deleting extra images (keeping only 2k images)
import os
import random
import shutil

def randomly_delete_images(directory_path, num_to_keep):
    # List all files in the directory
    files = os.listdir(directory_path)

    # Calculate the number of files to delete
    num_to_delete = len(files) - num_to_keep

    # Randomly select files to delete
    files_to_delete = random.sample(files, num_to_delete)

    # Delete selected files
    for file_name in files_to_delete:
        file_path = os.path.join(directory_path, file_name)
        os.remove(file_path)


# Example usage:
directory_path = destination_folder  # Change this to your directory path
num_to_keep = 100
randomly_delete_images(directory_path, num_to_keep)

# **Generating 100 Fake images**

In [ ]:
# !rm -rf /content/root/FAKE

In [ ]:
import os

save_folder = "/content/root/FAKE"
if not os.path.exists(save_folder):
    os.makedirs(save_folder)

num_images = 100

# Move the noise tensor to the same device as the generator model
noise = torch.randn(num_images, nz, 1, 1, device=device)

# Pass the noise through the generator to generate fake images
with torch.no_grad():
    generated_images = gen(noise).detach().cpu()

for i in range(generated_images.shape[0]):
    # Get the generated image
    image = generated_images[i].permute(1, 2, 0).numpy()  # Convert tensor to numpy array

    # Normalize the image pixel values to the range [0, 1]
    image = (image - image.min()) / (image.max() - image.min())

    # Save the generated image
    img_path = os.path.join(save_folder, f"generated_image_{i+1}.png")
    plt.imsave(img_path, image)


In [ ]:
plt.figure(figsize=(num_images, num_images))  # Adjust figure size as neede
for i in range(generated_images.shape[0]):
    plt.subplot(1, generated_images.shape[0], i + 1)
    plt.imshow(vutils.make_grid(generated_images[i], padding=2, normalize=True).permute(1, 2, 0))
    plt.axis('off')
    plt.title('{}'.format(i+1))

plt.show()

# **Cosine similarity**

In [ ]:
import torchvision.transforms as transforms
from torchvision.models import resnet18
from PIL import Image
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
model_rasnet = resnet18(pretrained=True)
model_rasnet.eval()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 159MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to match the expected input size of the model
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # Mean of ImageNet dataset
                         std=[0.229, 0.224, 0.225])   # Standard deviation of ImageNet dataset
])

In [ ]:
def extract_features(image_path):
    image = Image.open(image_path)
    image = transform(image).unsqueeze(0)  # Add batch dimension
    with torch.no_grad():
        features = model_rasnet(image)
    return features.squeeze().numpy()

In [ ]:
def compute_cosine_similarity(fake, real):
    fakes = os.listdir(fake)
    reals = os.listdir(real)

    features_set1 = [extract_features(os.path.join(fake, image_path)) for image_path in fakes]
    features_set2 = [extract_features(os.path.join(real, image_path)) for image_path in reals]

    similarity_matrix = cosine_similarity(features_set1, features_set2)
    return similarity_matrix

In [ ]:
fake = "/content/root/FAKE"
real = "/content/root/REAL"
sim_matrix = compute_cosine_similarity(real, fake)

RuntimeError: The size of tensor a (4) must match the size of tensor b (3) at non-singleton dimension 0

# **FID Score**

In [ ]:
! pip install pytorch-fid

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 67.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 701.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 760.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 2.6 MB/s eta 0:00:00


In [ ]:
# import shutil

# # Source and destination paths
# source_folder_positive = "/content/root/REAL"
# destination_folder = "/content/Root/REAL/real"

# # Move positive folder to root folder
# shutil.move(source_folder_positive, destination_folder)

'/content/Root/REAL/real'

In [ ]:
import os
import numpy as np
from torchvision.transforms import ToTensor
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

In [ ]:
def _compute_statistics_of_path(path, model, batch_size, dims, cuda):
    # Set device
    device = torch.device("cuda" if cuda and torch.cuda.is_available() else "cpu")

    # Define transformation
    transform = ToTensor()

    # Load data
    dataset = ImageFolder(root=path[0],transform=transform)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=4)


    # Initialize variables for computing mean and standard deviation
    mean = torch.zeros(dims)
    std = torch.zeros(dims)
    count = 0

    # Set model to evaluation mode
    model.eval()

    # Iterate through batches
    with torch.no_grad():
        for inputs in dataloader:
            # Move data to appropriate device
            inputs = inputs.to(device)

            # Forward pass
            outputs = model(inputs)

            # Compute mean and std incrementally
            batch_size_i = inputs.size(0)
            delta = outputs - mean
            mean += torch.sum(delta, dim=0) / count
            delta2 = outputs - mean
            std += torch.sum(delta * delta2, dim=0)
            count += batch_size_i

    # Compute final statistics
    mean /= count
    std = torch.sqrt(std / count)

    return mean, std


In [ ]:
from pytorch_fid.inception import InceptionV3
batch_size = 64
device = device
dims = 2048
real_images_path = "/content/Root/REAL"
fake_images_path = "/content/Root/FAKE"
def calculate_fid_given_paths(paths, batch_size, cuda, dims):
    """Calculates the FID of two paths"""
    for p in paths:
        if not os.path.exists(p):
            raise RuntimeError('Invalid path: %s' % p)

    block_idx = InceptionV3.BLOCK_INDEX_BY_DIM[dims]

    model = InceptionV3([block_idx])
    if cuda:
        model.cuda()

    m1, s1 = _compute_statistics_of_path(paths[0], model, batch_size, dims, cuda)
    m2, s2 = _compute_statistics_of_path(paths[1], model, batch_size, dims, cuda)
    fid_value = calculate_frechet_distance(m1, s1, m2, s2)
    return fid_value

In [ ]:
calculate_fid_given_paths([real_images_path, fake_images_path], batch_size, device, dims)

Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/queues.py", line 239, in _feed
    reader_close()
  File "/usr/lib/python3.10/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/usr/lib/python3.10/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/queues.py", line 239, in _feed
    reader_close()
  File "/usr/lib/python3.10/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/usr/lib/python3.10/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor

Du

KeyboardInterrupt: 

In [ ]:
# import os
# import numpy as np
# from torchvision.transforms import ToTensor
# from torchvision.datasets import ImageFolder
# from torch.utils.data import DataLoader

# # Define paths to the image folders
# real_images_path = "/content/Root/REAL"
# fake_images_path = "/content/Root/FAKE"

# # Transform to apply to images
# transform = ToTensor()

# # # Load real and fake datasets
# # real_dataset = ImageFolder(root=real_images_path, transform=transform)
# # fake_dataset = ImageFolder(root=fake_images_path, transform=transform)

# # # Define batch size for loading images
# # batch_size = 64

# # # Create data loaders
# # real_loader = DataLoader(real_dataset, batch_size=batch_size, shuffle=False)
# # fake_loader = DataLoader(fake_dataset, batch_size=batch_size, shuffle=False)
# batch_size = 64
# device = device
# dims = 2048
# transform = ToTensor()
# # Extract features and compute FID
# fid_score = calculate_fid_given_paths([real_images_path, fake_images_path],batch_size,device,dims)

# print("FID Score:", fid_score)


TypeError: calculate_frechet_distance() missing 2 required positional arguments: 'mu2' and 'sigma2'

In [ ]:
# import torch
# from torchvision.datasets import ImageFolder
# from torch.utils.data import DataLoader
# from pytorch_fid.inception import InceptionV3
# from pytorch_fid.fid_score import calculate_frechet_distance

# def calculate_fid_given_paths(paths, batch_size, device, dims):
#     # Set up data loaders
#     dataset = ImageFolder(root=paths[0],transform=transform)
#     dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=4)

#     dataset = ImageFolder(root=paths[1], transform=transform)
#     fake_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=4)

#     # Set up device
#     device = torch.device(device)

#     # Load pre-trained InceptionV3 model
#     block_idx = InceptionV3.BLOCK_INDEX_BY_DIM[dims]
#     inception_model = InceptionV3([block_idx]).to(device)

#     # Compute the activations
#     real_activations = compute_activations(dataloader, inception_model, device)
#     fake_activations = compute_activations(fake_dataloader, inception_model, device)

#     # Compute FID score
#     fid_score = calculate_frechet_distance(real_activations, fake_activations)

#     return fid_score

# def compute_activations(loader, model, device):
#     model.eval()
#     preds = []
#     with torch.no_grad():
#         for batch in loader:
#             images, _ = batch
#             images = images.to(device)
#             preds.append(model(images)[0].view(images.shape[0], -1))
#     return torch.cat(preds, dim=0)
